# Hardware Performance Testing

Test TinyMPC hardware performance by measuring execution time as a function of max_iter for different bitstreams

## 1. Setup and Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
import pandas as pd
from scipy import stats

# Add driver path
os.chdir("/home/xilinx/jupyter_notebooks/zhenyu/tinympc_ip_gen/")

sys.path.append('driver')
from tinympc_hw import tinympc_hw

# Import dynamics for test problem setup
from dynamics import LinearizedQuadcopterDynamics, CrazyflieParams, NoiseModel

print("All modules imported successfully")

In [ ]:
# Initialize dynamics model for test problem
params = CrazyflieParams()
noise_model = NoiseModel()
dynamics = LinearizedQuadcopterDynamics(params, noise_model)

# Generate system matrices
control_freq = 100.0  # Hz
A, B = dynamics.generate_system_matrices(control_freq)
Q, R = dynamics.generate_cost_matrices()
constraints = dynamics.generate_constraints()

# System dimensions (fixed for quadrotor)
nx = 12  # State dimension
nu = 4   # Control dimension

print(f"Test problem configured:")
print(f"  State dimension (nx): {nx}")
print(f"  Control dimension (nu): {nu}")
print(f"  Note: Prediction horizon (N) will be extracted from each bitstream")

In [ ]:
# Generate test data
def generate_test_data(nx, nu, N):
    """Generate random test data for MPC problem"""
    np.random.seed(42)  # For reproducibility
    
    # Initial state with some deviation from origin
    x0 = np.random.randn(nx) * 0.1
    x0[2] = 1.0  # Set altitude to 1m
    
    # Reference trajectory (hover at origin)
    xref = np.zeros((N, nx))
    xref[:, 2] = 1.0  # Reference altitude
    
    # Reference control (hover)
    uref = np.zeros((N-1, nu))
    
    return x0, xref, uref

def extract_N_from_bitstream(bitstream_path):
    """Extract N parameter from bitstream filename"""
    import re
    filename = bitstream_path.name if hasattr(bitstream_path, 'name') else str(bitstream_path)
    pattern = r'tinympcproj_N(\d+)_'
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Could not extract N from {filename}, using default N=5")
        return 5

# Note: Test data will be generated per bitstream with correct N
print("Test data generation functions defined")

In [ ]:
# Find all bitstream files in subdirectories
import os
from pathlib import Path
import glob

def find_all_bitstreams(base_path="."):
    """Find all .bit files in the project directory and subdirectories"""
    bitstream_files = []
    
    # Search for .bit files recursively
    search_pattern = os.path.join(base_path, "**", "*.bit")
    found_files = glob.glob(search_pattern, recursive=True)
    
    # Convert to Path objects and filter out any invalid paths
    for file_path in found_files:
        path_obj = Path(file_path)
        if path_obj.exists() and path_obj.is_file():
            bitstream_files.append(path_obj)
    
    # Also check specific known locations
    known_dirs = ["bitstream", "impl", "output", "build"]
    for dir_name in known_dirs:
        dir_path = Path(base_path) / dir_name
        if dir_path.exists() and dir_path.is_dir():
            bit_files = list(dir_path.glob("*.bit"))
            for bit_file in bit_files:
                if bit_file not in bitstream_files:
                    bitstream_files.append(bit_file)
    
    return sorted(bitstream_files)

def test_bitstream_performance(bitstream_path, test_maxiter_values=[10, 100, 1000], num_trials=10):
    """
    Test a bitstream with specific max_iter values, with warmup run
    
    Args:
        bitstream_path: Path to bitstream file
        test_maxiter_values: List of max_iter values to test (default: [10, 100, 1000])
        num_trials: Number of trials per max_iter value (excluding warmup)
    
    Returns:
        dict: Results with average execution times
    """
    # Extract N from bitstream filename
    N = extract_N_from_bitstream(bitstream_path)
    
    # Generate test data with correct dimensions
    nx = 12  # State dimension (fixed for quadrotor)
    nu = 4   # Control dimension (fixed for quadrotor)
    x0_test, xref_test, uref_test = generate_test_data(nx, nu, N)
    
    # Initialize hardware solver
    hw_solver = tinympc_hw(bitstream_path=str(bitstream_path))
    
    results = {
        'bitstream': str(bitstream_path),
        'N': N,
        'maxiter_values': test_maxiter_values,
        'avg_times': {},
        'all_times': {}
    }
    
    for max_iter in test_maxiter_values:
        # Set check_termination equal to max_iter as requested
        check_termination = max_iter
        hw_solver.setup(max_iter=max_iter, check_termination=check_termination, verbose=0)
        
        # Warmup run (first run to prepare hardware)
        hw_solver.set_x0(x0_test)
        hw_solver.set_x_ref(xref_test)
        hw_solver.set_u_ref(uref_test)
        hw_solver.solve(timeout=1.0)  # Warmup - don't record this time
        
        # Actual timing runs
        times = []
        for trial in range(num_trials):
            # Set problem data
            hw_solver.set_x0(x0_test)
            hw_solver.set_x_ref(xref_test)
            hw_solver.set_u_ref(uref_test)
            
            # Measure execution time
            start_time = time.perf_counter()
            success = hw_solver.solve(timeout=1.0)
            end_time = time.perf_counter()
            
            if success:
                exec_time = (end_time - start_time) * 1000  # Convert to ms
                times.append(exec_time)
        
        if len(times) > 0:
            results['avg_times'][max_iter] = np.mean(times)
            results['all_times'][max_iter] = times
    
    # Cleanup
    hw_solver.cleanup()
    
    return results

# Find all available bitstreams
print("Searching for bitstream files...")
bitstreams = find_all_bitstreams()

if len(bitstreams) == 0:
    print("No bitstream files found. Looking in current directory and subdirectories...")
    # Try with absolute path
    bitstreams = find_all_bitstreams("/home/xilinx/jupyter_notebooks/zhenyu/tinympc_ip_gen/")

print(f"\nFound {len(bitstreams)} bitstream file(s):")
for idx, bitstream in enumerate(bitstreams):
    print(f"  {idx+1}. {bitstream}")

# Select first bitstream as default if available
if len(bitstreams) > 0:
    selected_bitstream = bitstreams[0]
    print(f"\nDefault selected bitstream: {selected_bitstream}")

In [ ]:
# Test all bitstreams with max_iter = 10, 100, 1000
all_test_results = []
test_maxiter_values = [10, 100, 1000]

if len(bitstreams) > 0:
    print(f"\nTesting {len(bitstreams)} bitstream(s) with max_iter values: {test_maxiter_values}")
    print("=" * 80)
    
    for idx, bitstream in enumerate(bitstreams):
        print(f"\n[{idx+1}/{len(bitstreams)}] Testing: {bitstream.name}")
        print("-" * 40)
        
        try:
            # Run performance test with warmup
            results = test_bitstream_performance(
                bitstream, 
                test_maxiter_values=test_maxiter_values,
                num_trials=10  # 10 trials after warmup for each max_iter
            )
            
            all_test_results.append(results)
            
            # Display results immediately
            print(f"  N parameter: {results['N']}")
            print(f"  Average execution times (ms):")
            for max_iter in test_maxiter_values:
                if max_iter in results['avg_times']:
                    avg_time = results['avg_times'][max_iter]
                    std_time = np.std(results['all_times'][max_iter])
                    print(f"    max_iter={max_iter:4d}: {avg_time:8.3f} ± {std_time:.3f} ms")
                    
        except Exception as e:
            print(f"  ERROR: Failed to test bitstream - {e}")
    
    print("\n" + "=" * 80)
    print("TESTING COMPLETE")
    print("=" * 80)
    
else:
    print("No bitstreams found to test.")

# Create summary table of all results
if len(all_test_results) > 0:
    # Build DataFrame for summary
    summary_data = []
    for result in all_test_results:
        row = {
            'Bitstream': Path(result['bitstream']).name,
            'N': result['N'],
        }
        # Add average times for each max_iter value
        for max_iter in test_maxiter_values:
            if max_iter in result['avg_times']:
                row[f'max_iter={max_iter} (ms)'] = f"{result['avg_times'][max_iter]:.3f}"
            else:
                row[f'max_iter={max_iter} (ms)'] = "N/A"
        summary_data.append(row)
    
    df_summary = pd.DataFrame(summary_data)
    
    print("\nSUMMARY TABLE - Average Execution Times")
    print("=" * 80)
    print(df_summary.to_string(index=False))
    print("=" * 80)
    
    # Export to CSV
    csv_filename = 'bitstream_performance_summary.csv'
    df_summary.to_csv(csv_filename, index=False)
    print(f"\nResults exported to: {csv_filename}")

## Bitstream Switch Time Test

In [ ]:
import time
all_bitstreams = list(Path("bitstream").glob("*.bit"))

# Test bitstream switch time
print("\n" + "="*60)
print("BITSTREAM SWITCH TIME TEST")
print("="*60)

if len(all_bitstreams) < 2:
    print("Need at least 2 bitstreams to test switching time")
else:
    # Initialize results storage
    switch_times = []
    
    # Number of switches to test
    num_tests = 10
    
    print(f"Testing {num_tests} switches between bitstreams...")
    
    # Alternate between first two bitstreams
    for i in range(num_tests):
        bitstream = all_bitstreams[i % len(all_bitstreams)]
        
        # Record start time
        start_time = time.time()
        new_solver = tinympc_hw(bitstream_path=str(bitstream))
        switch_time = (time.time() - start_time) * 1000  # Convert to ms
        switch_times.append(switch_time)
        print(f"Switch {i+1}: {switch_time:.2f} ms")
    
    # Calculate statistics
    avg_switch = np.mean(switch_times)
    std_switch = np.std(switch_times)
    
    print("\nResults:")
    print(f"Average switch time: {avg_switch:.2f} ms")
    print(f"Standard deviation: {std_switch:.2f} ms")
    print("="*60)
